In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# change this to point to wherever you've put the data
data_dir = "F:\\GoogleDrive\\ds_ml\\RIT\\2022-01 DSCI.644\\project\\phase2\\"  # on Ryzen workstation
# data_dir = "D:\\GoogleDrive\\ds_ml\\RIT\\2022-01 DSCI.644\\project\\phase2\\"  # on Xeon workstation
data_path = data_dir + "project3-authors.csv"
df_raw = pd.read_csv(data_path)
# df_raw.shape   # (17317, 9)
# df_raw.columns # ['Name', 'CommitId', 'RefactoringType', 'FilePath', 'Class',
#                   'RefactoringDetail', 'Message', 'AuthorName', 'AuthorEmail']
# df_raw['Name'].unique()  # ['hibernate$hibernate-validator', 'eclipse$bpmn2-modeler',
#                             'adangel$pmd']
df_raw.head()

,Name,CommitId,RefactoringType,FilePath,Class,RefactoringDetail,Message,AuthorName,AuthorEmail
0,hibernate$hibernate-validator,a3af1683e63c9c800f486552e7e5002c24b6712b,Move Source Folder,src,NaN,Move Source Folder\tsrc to hibernate-validator...,Moved the legacy validator code into hibernate...,Hardy Ferentschik,hibernate@ferentschik.de
1,hibernate$hibernate-validator,6d6c3deaf0b499e06077383d8d3add43f30ba099,Move Source Folder,hibernate-validator-legacy/src,NaN,Move Source Folder\thibernate-validator-legacy...,Refactored build so that compile and test can ...,Hardy Ferentschik,hibernate@ferentschik.de
2,hibernate$hibernate-validator,6d6c3deaf0b499e06077383d8d3add43f30ba099,Move Source Folder,hibernate-validator-legacy/src/test,NaN,Move Source Folder\thibernate-validator-legacy...,Refactored build so that compile and test can ...,Hardy Ferentschik,hibernate@ferentschik.de
3,hibernate$hibernate-validator,25350de39ef9484997999f6e6d8f00abc5d51546,Extract Variable,hibernate-validator/src/main/java/org/hibernat...,org.hibernate.validation.engine.ValidatorImpl,Extract Variable\tleafBeanInstance : Object in...,BVAL-37 ConstraintDescriptor.getLeafValue()\n\...,Emmanuel Bernard,emmanuel@hibernate.org
4,hibernate$hibernate-validator,25350de39ef9484997999f6e6d8f00abc5d51546,Rename Method,hibernate-validator/src/main/java/org/hibernat...,org.hibernate.validation.engine.ValidatorImpl,Rename Method\tpublic getBeanConstraints() : E...,BVAL-37 ConstraintDescriptor.getLeafValue()\n\...,Emmanuel Bernard,emmanuel@hibernate.org


In [2]:
df_raw.columns

Index(['Name', 'CommitId', 'RefactoringType', 'FilePath', 'Class',
       'RefactoringDetail', 'Message', 'AuthorName', 'AuthorEmail'],
      dtype='object')

In [3]:
# filter for just the adangel$pmd rows
df_adangle = df_raw[df_raw['Name'] == "adangel$pmd"].copy()

# filter for just the hibernate$hibernate-validator rows
df_hiber = df_raw[df_raw['Name'] == "hibernate$hibernate-validator"].copy()

# df_hiber.shape  # (5399, 9) same row count as report in phase 1 repoort, check
# df_adangle.shape  # (8495, 9) same row count as report in phase 1 repoort, check

# reset the indices
# df_hiber.reset_index().drop(['index'], axis=1)  # didn't do anything
df_hiber.tail()

,Name,CommitId,RefactoringType,FilePath,Class,RefactoringDetail,Message,AuthorName,AuthorEmail
5394,hibernate$hibernate-validator,9e0626b8318ba4bdf55580aacae3947410278129,Extract Variable,engine/src/main/java/org/hibernate/validator/i...,org.hibernate.validator.internal.engine.Valida...,Extract Variable\tbeanMetaData : BeanMetaData<...,HV-1681 Separate out bean metadata aware Value...,marko-bekhta,marko.prykladna@gmail.com
5395,hibernate$hibernate-validator,9e0626b8318ba4bdf55580aacae3947410278129,Move Class,engine/src/main/java/org/hibernate/validator/i...,org.hibernate.validator.internal.engine.ValueC...,Move Class\torg.hibernate.validator.internal.e...,HV-1681 Separate out bean metadata aware Value...,marko-bekhta,marko.prykladna@gmail.com
5396,hibernate$hibernate-validator,59372ef6b92138b0c6a347ea16cd655576715252,Rename Variable,engine/src/main/java/org/hibernate/validator/i...,org.hibernate.validator.internal.engine.valuee...,Rename Variable\textractorDescriptorsToCache :...,HV-1684 Fix value extraction logic to avoid st...,Guillaume Smet,guillaume.smet@gmail.com
5397,hibernate$hibernate-validator,59372ef6b92138b0c6a347ea16cd655576715252,Rename Variable,engine/src/main/java/org/hibernate/validator/i...,org.hibernate.validator.internal.engine.valuee...,Rename Variable\textractorDescriptorsToCache :...,HV-1684 Fix value extraction logic to avoid st...,Guillaume Smet,guillaume.smet@gmail.com
5398,hibernate$hibernate-validator,571ab96b9ea2b32c2597f0765f59da28f5c029d9,Rename Method,engine/src/main/java/org/hibernate/validator/i...,org.hibernate.validator.internal.engine.path.P...,Rename Method\tpublic getPathWithoutLeafNode()...,HV-1691 Rename PathImpl#getPathWithoutLeafNode...,Guillaume Smet,guillaume.smet@gmail.com


## Remove records with more than one filepath

Both the adangle and hibernate projects had a small number of records that contained references to more than a single filepath.  Since these records constitute < 1% of the adangle records, we decided to remove them instead of trying coming up with a scheme to process them.

In [4]:
# How many records have FilePath entries containing more than one path?
# df_multi_paths_ada = df_adangle[df_adangle['FilePath'].str.contains(',')]
# df_multi_paths_hib = df_hiber[df_hiber['FilePath'].str.contains(',')]
# df_multi_paths_ada.shape, df_multi_paths_hib.shape  # 80/8495 ada, 19/5399 hiber, <1% so remove them
df_ada_singles = df_adangle[df_adangle['FilePath'].str.contains(',') == False]
df_hib_singles = df_hiber[df_hiber['FilePath'].str.contains(',') == False]

In [5]:
# df_paths.to_csv("paths.csv", header=False, index=False)

In [6]:
df_ada_unparsed_filepath = df_ada_singles[['RefactoringType', 'FilePath', 'AuthorName']].copy()
df_hib_unparsed_filepath = df_hib_singles[['RefactoringType', 'FilePath', 'AuthorName']].copy()
# df_ada_unparsed_filepath.shape, df_hib_unparsed_filepath.shape  # ((8415, 3), (5380, 3))

## Author Groupings

Interestingly, the same commit buckets used for the adangel project work fine for the hibernate project.  From the Phase3Explore notebook, we see the following counts in these buckets:

+  3 had > 400 commits  (very high frequency committers)
+  3 had > 100 and <= 400 commits  (high frequency committers)
+  4 had  > 25 and <= 100 commits  (medium frequency committers)
+ 12 had <= 25 commits  (low frequency committers)

## Fix Author issue in hibernate data

There are two authors in the hibernate projects that cause issues if not addressed.
1. **marko bekhta** is listed in two different ways in the data: marko.bekhta = marko-bekhta = marko bekhta
2. The apostrophe in David D'Alto

To fix issue 1., convert both forms to **marko bekhta**.  To fix issue 2., the apostrophe was removed.

After fixing these issues, we put the authors into the buckets define above.

In [7]:
# hibernate issue: fix the marko.bekhta = marko-bekhta = marko bekhta issue
df_hib_unparsed_filepath['AuthorName'] = \
    df_hib_unparsed_filepath['AuthorName'].str.replace('.', ' ', regex=False)
df_hib_unparsed_filepath['AuthorName'] = \
    df_hib_unparsed_filepath['AuthorName'].str.replace('-', ' ', regex=False)
# fix Davide D'Alto
df_hib_unparsed_filepath['AuthorName'] = \
    df_hib_unparsed_filepath['AuthorName'].str.replace("'", "", regex=False)

In [8]:
# get the committers for each bucket so we can filter on them

## df_authors - a pandas df with an AuthorName column
## 
def get_committer_buckets(df_authors, ex_high_bucket=400, high_bucket=100, med_bucket=25):
    authors = df_authors['AuthorName'].value_counts()
    committers_extremely_high = authors[authors > ex_high_bucket].index.to_list()
    committers_high = authors[(authors > high_bucket) & (authors <= ex_high_bucket)].index.to_list()
    committers_medium = authors[(authors > med_bucket) & (authors <= high_bucket)].index.to_list()
    committers_low = authors[(authors < med_bucket)].index.to_list()
    
    return committers_extremely_high, committers_high, committers_medium, committers_low
    
author_buckets_ada = get_committer_buckets(df_ada_unparsed_filepath)
author_buckets_hib = get_committer_buckets(df_hib_unparsed_filepath)
# author_buckets_ada[0], author_buckets_ada[1], author_buckets_ada[2], author_buckets_ada[3]
# author_buckets_hib[0], author_buckets_hib[1], author_buckets_hib[2], author_buckets_hib[3]

In [11]:
type(df_hib_unparsed_filepath)

pandas.core.frame.DataFrame

In [9]:

def create_author_group(df_authors, committers_extremely_high, committers_high, committers_medium):
    '''Adds an AuthorsGroup column to the input dataframe
    
    Args:
        df_authors (pandas.core.frame.DataFrame): Dataframe that contains a AuthorName column
        committers_extremely_high (list): list of names of the extremely high committers
        committers_high (list): list of names of the high committers
        committers_medium (list): list of names of the medium committers
    
    Returns:
        pandas.core.frame.DataFrame: input dataframe with AuthorName column added
    
    '''
    df_authors['AuthorGroup'] = \
    np.where(df_authors['AuthorName'].isin(committers_extremely_high),
         "extremely high",
         np.where(df_authors['AuthorName'].isin(committers_high),
                  "high", np.where(df_authors['AuthorName'].isin(committers_medium),
                  "medium", "low")))
    return df_authors
    

# df_design_matrix_unparsed_filepath['AuthorGroup'] = \
# np.where(df_design_matrix_unparsed_filepath['AuthorName'].isin(committers_extremely_high),
#          "extremely high",
#          np.where(df_design_matrix_unparsed_filepath['AuthorName'].isin(committers_high),
#                   "high", np.where(df_design_matrix_unparsed_filepath['AuthorName'].isin(committers_medium),
#                   "medium", "low")))

df_ada = create_author_group(df_ada_unparsed_filepath, author_buckets_ada[0],
                             author_buckets_ada[1], author_buckets_ada[2])

df_hib = create_author_group(df_hib_unparsed_filepath, author_buckets_hib[0],
                             author_buckets_hib[1], author_buckets_hib[2])

# check each AuthorGroup for adangle
# df_ada[df_ada['AuthorName'].isin(author_buckets_ada[0])]  # 5940
# df_ada[df_ada['AuthorName'].isin(author_buckets_ada[1])]  # 1772
# df_ada[df_ada['AuthorName'].isin(author_buckets_ada[2])]  #  388
# df_ada[df_ada['AuthorName'].isin(author_buckets_ada[3])]  #  315
#                                                           # 8415 total, check

In [31]:
# Use google docstring style to doc function:
# https://realpython.com/documenting-python-code/#documenting-your-python-code-base-using-docstrings
def get_shared_prop(df, left_group = 'low', right_group = 'extremely high'):
    '''Computes the proportion of files committed by the left_group that were
    also committed by the right_group.
    
    Args:
        df (pandas.core.frame.DataFrame): dataframe with AuthorGroup and FilePath columns. The
          AuthorGroup column is expected to contains values defined by left_group and right_group.
        left_group (str): left group of committers for which proportion is being computed
        right_group (str): right group of committers to check against left group
    
    Returns:
        tuple: 3-tuple where
        tuple[0] = proportion of left_group files committed that are shared right_group commits,
        tuple[1] = number of left_group files committed that are shared right_group commits, and
        tuple[2] = total number of left_group files committed
    '''
    
    df_right = df[df['AuthorGroup'] == right_group].copy()
    df_right['right_row'] = df_right.index     # keep track of the rows of the right group
    df_left = df[df['AuthorGroup'] == left_group].copy()
    df_left['left_row'] = df_left.index        # keep track of the rows of the left group
    df_left_merge = df_left.merge(df_right, on='FilePath', how='left').copy()
    df_left_merge_trimmed = df_left_merge[['FilePath', 'AuthorGroup_x', 'left_row', 'right_row']]
    # want intersection, so remove all the rows that were committed by the left_group but NOT the right
    df_left_intersect = df_left_merge_trimmed.dropna(subset = ['right_row']).copy()
    # If right_group larger, merge can create multiple rows per same left row. Remove these duplicates:
    df_left_intersect.drop('right_row', axis=1, inplace=True)
    df_left_intersect.drop_duplicates(inplace=True)
    
    return df_left_intersect.shape[0], df_left.shape[0], df_left_intersect.shape[0] / df_left.shape[0]

# check low vs extremely high
ada_low_props = get_shared_prop(df_ada)
hib_low_props = get_shared_prop(df_hib)
print(ada_low_props, hib_low_props)  # (115, 315, 0.36508) (43, 63, 0.68254) ~2x more overlap with hib

(115, 315, 0.36507936507936506) (43, 63, 0.6825396825396826)


In [32]:
# check medium vs extremely high
ada_med_props = get_shared_prop(df_ada, 'medium')
hib_med_props = get_shared_prop(df_hib, 'medium')
print(ada_med_props, hib_med_props)  # (96, 388, 0.2474) (78, 183, 0.4262) also ~2x more overlap with hib

(96, 388, 0.24742268041237114) (78, 183, 0.4262295081967213)


In [ ]:
# test of proportions, H0: p_match_hib = p_match_ada, HA: p_match_hib > p_match_ada

In [ ]:
# write the data to files so they can be read in and analyzed later
# df_design_matrix.to_csv("df_design_matrix_hibernate.csv", header=True, index=False)